# A rigorous, thorough, and high quality determination of phase transition existence and critical temperature in the 2D Ising Model

Determine the phase transition exists:
1. Mag vs. Temp
2. Sus. vs. Temp
3. Heat cap vs. Temp
4. Autocorrelation vs. Temp

Determine that the data we are taking is independent:
1. Autocorrelation as a func of lag time
2. Mag/sus/heat time series data
3. ?

Calculate critical temperature:
1. Binder cumulant
2. Finite scaling techniques??????
3. autocor?????

Do these things for various lattice sizes (can I extrapolate to infinite lattice size?). Also think about error bars/uncertainty for each method and lattice size

### Imports

In [2]:
#imports 
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy import optimize
import json
import copy
from scipy.optimize import curve_fit
import math
from scipy import fft

### Lattice Algorithms

In [9]:
def initLattice(latticeSize: int, hot: bool) -> list[list[int]]:
    lattice = np.zeros((latticeSize, latticeSize))
    if hot:
        for i in range(latticeSize):
            for j in range(latticeSize):
                #pick a random spin
                spin = 0
                randomInt = random.randint(0,1)
                if(randomInt == 1):
                    spin = 1
                else:
                    spin = -1
                #set lattice site equal to the random spin    
                lattice[i][j] = spin
    else: #lattice is cold
        for i in range(latticeSize):
            for j in range(latticeSize):
                #set all lattice sites to spin up
                spin = 1
                lattice[i][j] = spin
    
    return lattice

In [10]:
# Calculate change in energy of lattice by flipping a single site (i,j)
def deltaU(i: int, j: int, lattice: list) -> float:
    '''
    This calulation requires considering neighboring sites (first term in Hamiltonian)
    Therefore, we will use periodic boundary conditions (torus)
    I would like to imlement the external field term so you can drive the system to specific states

    E1 = -spin(i,j)*sum(spin(neighbors)),     E2 = spin(i,j)*sum(spin(neighbors))
    Ediff = E2 - E1 = 2spin(i,j)*sum(neighbors) (if spin(i,j) is 1 (up))            <<<<< NO epsilon/J? unclear why, currently just implementing pseudocode exactly as written

    In the mean field approximation E_up = -4J*sum(spin(neighbors))/4) 

    i is vertical, j is horizontal, zero indexed
    '''

    size = len(lattice)
    # If site is in an edge, apply periodic boundary conditions
    if(i == 0):
        top = lattice[size-1,j]
    else:
        top = lattice[i-1][j]
    if(i == size-1):
        bottom = lattice[0][j]
    else:
        bottom = lattice[i+1][j]
    if(j == 0):
        left = lattice[i][size-1]
    else:
        left = lattice[i][j-1]
    if(j == size-1):
        right = lattice[i][0]
    else:
        right = lattice[i][j+1]

    #now calculate the energy difference
    Ediff = 2*lattice[i][j]*(top+bottom+left+right)
    return Ediff

In [114]:
def Metropolis(lattice, temp, iterations):
    m = []
    for iteration in range(iterations):

        if((iteration % 99 == 0) and (iteration != 0)):
            sum = 0
            for xSite in range(lattice[0].size):
                for ySite in range(lattice[0].size):
                    sum += lattice[xSite][ySite]
            m.append(abs(sum)) #NEW: try storing abs of mag

        i = random.randint(0,lattice[0].size-1)
        j = random.randint(0,lattice[0].size-1)
        Ediff = deltaU(i,j,lattice)
        #Metropolis to decide whether site should be flipped. Needs to be iterated 100 times??
        if(Ediff <= 0):
            lattice[i][j] = -lattice[i][j] 
        else:
            #now only flip site according to Boltzmann factor
            boltzmannRandom = random.uniform(0,1)
            if(boltzmannRandom < np.exp(-Ediff/temp)): #Ediff must be positive so exponential is between 0 and 1
                lattice[i][j] = -lattice[i][j]

    return m

### Observable plots
Things mentioned in the first sections go here.

In [128]:
#Magnetization vs temperature
lattice = initLattice(12, True)
T = 10

magnetizationArray = []
numMeasurements = 10

for measurement in np.arange(numMeasurements):
    curMag = Metropolis(lattice, T, 500)
    magnetizationArray.append(curMag)

#flatten array
magnetizationArray = np.array(magnetizationArray)
magnetizationArray = magnetizationArray.flatten()
magnetizationArray

array([ 4.,  4.,  0., 18., 22.,  4., 22., 32., 10.,  4., 10.,  2., 16.,
        2.,  4., 16.,  6.,  6.,  6.,  4.,  6., 18.,  6.,  0., 18.,  0.,
       16., 18., 16.,  4., 10.,  6., 24.,  6., 18.,  4., 12.,  6.,  6.,
       24., 24., 20., 16., 16.,  4., 12.,  8.,  8., 22., 30.])

In [121]:
magnetizationArray

[[24.0]]